<a href="https://colab.research.google.com/github/a-forty-two/HPE-DecJanFeb2021/blob/main/HPE_08_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#ACTIVATION -> filters, transforms
# input_signal ----(activation)---- output_signal
# output_signal -> -1 to 1-> tanh
# probaility-> 0 to 1 -> Sigmoid 
# probab_distribution-> [0.8,0.2] -> softmax
# only Pos-> ReLU
# neg_num (tending to 0)-> LeakyRELU 

In [2]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')
from sklearn.preprocessing import LabelBinarizer # Label encoding, 1-hot encoding, multi-encoding
# LABEL binarizer is a 1-hot encoded MATRIX 
import cv2
import numpy as np
import random
import imutils
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from sklearn.model_selection import train_test_split

In [3]:
!ls data

cat  dog


In [4]:
# base url + 0000...9999 + '.png'
# 0000.png to 9999.png 
HP_dataset = 'data'
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'
HP_test_dataset = 'test'
HP_epoch = 100
HP_init_lr = 1e-3 # learning_rate = 0.001
HP_batch_size = 32
HP_image_dim = (96,96,3)


In [5]:
data = []
labels = [] 
# read all images
all_images = sorted(list(paths.list_images(HP_dataset)))
all_images[:5]

['data/cat/00.jpg', 'data/cat/01.jpg', 'data/dog/02.jpg', 'data/dog/03.jpg']

In [6]:
random.seed(42)
random.shuffle(all_images)
all_images[:5]

['data/dog/02.jpg', 'data/cat/01.jpg', 'data/dog/03.jpg', 'data/cat/00.jpg']

In [7]:
# data/ [class_name] / [image_name ].extension 
#                  split by (/, //, \) -> os.separators
import os
for impath in all_images:
  img = cv2.imread(impath)
  resized = cv2.resize(img, (HP_image_dim[0],HP_image_dim[1]) )
  imageData = img_to_array(resized)
  data.append(imageData)
  # extract label from filename (2nd last element) / \\ 
  label = impath.split(os.path.sep)[-2]
  labels.append(label)



In [8]:
data[0]

array([[[103., 135., 158.],
        [101., 134., 157.],
        [101., 133., 156.],
        ...,
        [222., 214., 206.],
        [216., 203., 197.],
        [203., 187., 186.]],

       [[102., 134., 157.],
        [101., 135., 157.],
        [103., 135., 158.],
        ...,
        [163., 164., 173.],
        [164., 161., 171.],
        [203., 186., 186.]],

       [[105., 136., 159.],
        [103., 136., 159.],
        [104., 136., 159.],
        ...,
        [161., 167., 178.],
        [162., 165., 177.],
        [162., 166., 175.]],

       ...,

       [[113., 145., 168.],
        [114., 146., 169.],
        [112., 144., 167.],
        ...,
        [ 29.,  52.,  84.],
        [ 39.,  67., 102.],
        [ 30.,  57.,  91.]],

       [[110., 142., 165.],
        [110., 142., 165.],
        [112., 143., 166.],
        ...,
        [ 26.,  51.,  88.],
        [ 40.,  70., 105.],
        [ 30.,  61.,  94.]],

       [[111., 142., 166.],
        [111., 144., 165.],
        [109., 1

In [10]:
# LABELS 
# 0-1-> Cat and DOG-> [0 1] , [1 0]
data = np.array(data, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels

array([[1],
       [0],
       [1],
       [0]])

In [11]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(data, labels, test_size=0.2, random_state=42)


In [13]:
len(testx)

1

In [14]:
# Augmentation
aug = ImageDataGenerator( rotation_range=25, width_shift_range=0.1, 
                         height_shift_range=0.1, shear_range=0.2,
                         zoom_range=0.2, horizontal_flip=True,fill_mode='nearest')


In [17]:
from keras import backend
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
#N= 5
HP_block1_conv_dim = 32
HP_block2_conv_dim = 64
HP_block3_conv_dim = 128
HP_block4_conv_dim = 256
HP_block5_dense_dim = 1024
HP_small_pattern = (3,3) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (2,2) # 4 times compressed images from previous MP layers!!!
HP_dropout_little =0.25
HP_dropout_big = 0.50
# HP_epochs, batch_size-> are now problems of the developer USING this model. 

HP_img_dims = (96,96,3)


In [21]:
class RacoonVGG:
  @staticmethod
  def build(height, width, depth, classes):
    # assume that we are on TF, but if something else is detected, switch the dimension
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    model = Sequential()
    
    # BLOCK1
    model.add(Conv2D(HP_block1_conv_dim,HP_small_pattern, padding='same',
                     input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_small_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 2
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block2_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))
    
    # COMPLEX BLOCK 3
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block3_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # COMPLEX BLOCK 4
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(Conv2D(HP_block4_conv_dim,HP_small_pattern, padding='same'))
    model.add(Activation('relu'))
    model.add(BatchNormalization(axis=channel_dim))
    model.add(MaxPooling2D(pool_size=HP_large_pattern))
    model.add(Dropout(HP_dropout_little))

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())
    model.add(Dense(HP_block5_dense_dim))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(HP_dropout_big))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


    # TF-> (h,w,d)
    # Theano-> (d,h,w)
    # GENERIC PROGRAM-> support BOTH the methods!!! 
    # if using KERAS-> because KERAS is just an API that is front facing an unknown
    # backend!

In [22]:
from tensorflow.keras.optimizers import Adam
model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1], depth=HP_img_dims[2],classes = len(lb.classes_))
opt = Adam(lr=HP_init_lr, decay = HP_init_lr/ HP_epoch)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


# 0.1 ; 0.1 - 0.1/50; (0.1-0.1/50)- 0.1/50..... 

In [27]:
len(trainx)

3

In [23]:
hist = model.fit(aug.flow(trainx, trainy, batch_size=HP_batch_size),
                 validation_data=(testx, testy),
                 steps_per_epoch= len(trainx) // HP_batch_size,
                 epochs=HP_epoch, verbose=1)

Epoch 1/100


ValueError: ignored